In [145]:
# load the BTC long term dataset
import pandas as pd

btc_5y_df = pd.read_csv('../backend/data/btcusdt_1d.csv', index_col=0, parse_dates=True)

print(btc_5y_df.head().to_markdown())

| timestamp           | symbol   |    open |    high |   low |   close |   volume |
|:--------------------|:---------|--------:|--------:|------:|--------:|---------:|
| 2020-04-20 00:00:00 | BTCUSDT  | 7121.4  | 7220    |  6751 | 6826.83 |  90149.5 |
| 2020-04-21 00:00:00 | BTCUSDT  | 6828.98 | 6940    |  6762 | 6841.37 |  60109.7 |
| 2020-04-22 00:00:00 | BTCUSDT  | 6841.36 | 7156.38 |  6818 | 7125.14 |  61486.4 |
| 2020-04-23 00:00:00 | BTCUSDT  | 7125.12 | 7738    |  7020 | 7482.39 | 102774   |
| 2020-04-24 00:00:00 | BTCUSDT  | 7483.96 | 7615.96 |  7388 | 7505    |  60182.1 |


In [ ]:
# close prices
btc_5y_close_df = btc_5y_df.loc[:, 'close']

print("\n--- Setting up Data Split ---")

# Ensure data is sorted by time
btc_5y_close_df = btc_5y_close_df.sort_index()
btc_5y_close_df = btc_5y_close_df.to_frame()

# Create lag features
for lag in range(1, 3):  # Lags from 1 to 2 days
    btc_5y_close_df[f'lag_{lag}'] = btc_5y_close_df['close'].shift(lag)

# Drop NaN values caused by lagging
btc_5y_close_df = btc_5y_close_df.dropna()

# Restore the 'close' column name
# btc_5y_close_df = btc_5y_close_df.rename(columns={'close': 'close'})
btc_5y_close_df.head()


--- Setting up Data Split ---


,close,lag_1,lag_2
timestamp,,,
2020-04-22,7125.14,6841.37,6826.83
2020-04-23,7482.39,7125.14,6841.37
2020-04-24,7505.00,7482.39,7125.14
2020-04-25,7538.67,7505.00,7482.39
2020-04-26,7693.10,7538.67,7505.00


In [147]:
# Train-Test Split (last 365 days as test set)
split_date = btc_5y_close_df.index[-30]  
btc_train = btc_5y_close_df.loc[btc_5y_close_df.index <= split_date].copy()
btc_test = btc_5y_close_df.loc[btc_5y_close_df.index > split_date].copy()

btc_train.head()


,close,lag_1,lag_2
timestamp,,,
2020-04-22,7125.14,6841.37,6826.83
2020-04-23,7482.39,7125.14,6841.37
2020-04-24,7505.00,7482.39,7125.14
2020-04-25,7538.67,7505.00,7482.39
2020-04-26,7693.10,7538.67,7505.00


In [ ]:
import pandas as pd
from pycaret.regression import *

# PyCaret Regression Setup
xgb_exp = RegressionExperiment().setup(
    data=btc_train, 
    target="close",
    session_id=123, 
    fold=3,  # K-fold cross-validation
    data_split_shuffle=False,  # **Important: Keeps time-series order**
    fold_strategy="timeseries",  # Ensures time-series split
)

# Train XGBoost Model
xgb_model = xgb_exp.create_model('xgboost')



,Description,Value
0,Session id,123
1,Target,close
2,Target type,Regression
3,Original data shape,"(1794, 3)"
4,Transformed data shape,"(1794, 3)"
5,Transformed train set shape,"(1255, 3)"
6,Transformed test set shape,"(539, 3)"
7,Numeric features,2
8,Preprocess,True
9,Imputation type,simple


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,2938.3538,14330692.0000,3785.5901,0.8402,0.0746,0.0587
1,1338.6779,2871917.7500,1694.6733,0.9695,0.0639,0.0507
2,760.8511,1206861.2500,1098.5724,0.9399,0.0435,0.0298
Mean,1679.2943,6136490.3333,2192.9453,0.9165,0.0607,0.0464
Std,921.0118,5833912.8666,1152.1638,0.0553,0.0129,0.0122


In [149]:
# check the performance of the baseline model using the train set (not the validation data held off in the earlier step)

y_predict = xgb_exp.predict_model(xgb_model)


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Extreme Gradient Boosting,9053.9287,243341152.0000,15599.3955,0.4389,0.1972,0.1044


In [204]:
import plotly.express as px

fig = px.line(y_predict, y=['close', 'prediction_label'], template='plotly_dark', labels={"value" : "close price $"})

fig.show()

# fig.show()

In [203]:
tuned_xgb = xgb_exp.tune_model(xgb_model)

y_pred_tuned = xgb_exp.predict_model(tuned_xgb)

fig = px.line(y_pred_tuned, y=['close', 'prediction_label'], template='plotly_dark', labels={"value" : "close price $"})

fig.show()


Fitting 3 folds for each of 10 candidates, totalling 30 fits


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Extreme Gradient Boosting,9142.5830,250072528.0000,15813.6836,0.4234,0.2005,0.1052


In [152]:
# Finalize Model and Make Predictions
final_xgb = xgb_exp.finalize_model(tuned_xgb)
# predictions = predict_model(final_xgb, data=btc_test)
y_pred_final = xgb_exp.predict_model(final_xgb)

fig = px.line(y_pred_final, y=['close', 'prediction_label'], template='plotly_dark', labels={"value" : "close price $"})

fig.show()
# pull()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Extreme Gradient Boosting,1209.2119,2860261.0000,1691.2306,0.9934,0.0252,0.0186


In [153]:
# compare with furture data N.B there's no need to drop!! see next cell
y_pred_future = xgb_exp.predict_model(final_xgb, data=btc_test.drop('close', axis=1))

print(xgb_exp.pull())

train_forecast_df = pd.concat([y_pred_final, y_pred_future])
# Insert the last 30 days of the test set into the close column of train_forecast_df
train_forecast_df.loc[btc_test.index, 'close'] = btc_test['close']
fig = px.line(train_forecast_df, y=['close', 'prediction_label'], template='plotly_dark', labels={"value" : "close price $"})
fig.add_vrect(x0=btc_test.index[0], x1=btc_test.index[-1], fillcolor="grey", opacity=0.25, line_width=0)

fig.show()

                       Model          MAE        MSE         RMSE      R2  \
0  Extreme Gradient Boosting  1209.211914  2860261.0  1691.230591  0.9934   

    RMSLE    MAPE  
0  0.0252  0.0186  


In [171]:
# compare with furture data
y_pred_future = xgb_exp.predict_model(final_xgb, data=btc_test)

train_forecast_df = pd.concat([y_pred_final, y_pred_future])
# Insert the last 30 days of the test set into the close column of train_forecast_df
# train_forecast_df.loc[btc_test.index, 'close'] = btc_test['close']

fig = px.line(train_forecast_df, y=['close', 'prediction_label'], template='plotly_dark', labels={"value" : "close price $"})
fig.add_vrect(x0=btc_test.index[0], x1=btc_test.index[-1], fillcolor="grey", opacity=0.25, line_width=0)


fig.show()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Extreme Gradient Boosting,1959.1750,5594254.0000,2365.2175,0.1518,0.0285,0.0236


# transforming the data?

from the prior time series decomposition, attemping to transform the data didn't make the data more normally distributed. 


# observations

making predictions with lagged values will be require recursive forecasting where exogenous future variables are not available. In the case of lagged features, we can use the last predicted value and the next lagged value as our exogenous variables for prediciton.

In [155]:
def recursive_forecast(experiment: RegressionExperiment, model, data: pd.DataFrame, fh=30, lags=2):
    # get the last known values from the dataset based on the lags
    if len(data) > lags:
        last_known_df = list(data['close'].values[-lags:])
    else:
        last_known_df = list(data['close'].values)
    dates = []
    predictions = []
    
    # get the latest date (should be a timestamp so that timedelta works!)
    last_date = data.index[-1]
    # for window in forecast horizon - last_known_df will be the rolling window
    for window in range(fh):
        # prepare prediction input with correct lag structure i.e last known close data (reversed), columns are now lags, index is timedelta days + window
        input_df = pd.DataFrame([last_known_df[-lags:][::-1]], columns=[f"lag_{j+1}" for j in range(lags)], index=[last_date + pd.Timedelta(days=window+1)])
        print(input_df.to_markdown())
        # make prediction and get first value
        prediction = experiment.predict_model(model, data=input_df)
        prediction = prediction['prediction_label'].values[0]
        # append predictions, update history and dates (with the index of the latest input_df)
        predictions.append(prediction)
        last_known_df.append(prediction)
        dates.append(input_df.index[0])
    # return the prediction df {predictions:prediciton} index=dates
    predictions_df = pd.DataFrame(data=dict(prediction_label=predictions), index=dates)
    return predictions_df
    

def predict_and_plot(model, data:pd.DataFrame, plot=True):
    recursive_forecast(model, data)
    # combine true values and predictions for vis
    # y_pred_final_diff = y_predict_original.rename(columns={"y_pred": "close"})
    y_true_pred_df = pd.concat([btc_5y_close_df['close'].iloc[-len(y_predict):].to_frame(),
                                        y_predict], axis=0, copy=True)
   
    if plot:
        fig = px.line(y_true_pred_df, y=['close', 'prediction_label'], template='plotly_dark', labels={"value": "close price $"})
        fig.add_vrect(x0=data.index[0], x1=data.index[-1], fillcolor="grey", opacity=0.25, line_width=0)
        fig.show()



In [156]:
recursive_forecast(xgb_exp, final_xgb, data=btc_5y_close_df.iloc[-2:], fh=5)


|                     |   lag_1 |   lag_2 |
|:--------------------|--------:|--------:|
| 2025-04-19 00:00:00 | 84586.3 | 84947.9 |


|                     |   lag_1 |   lag_2 |
|:--------------------|--------:|--------:|
| 2025-04-20 00:00:00 | 86190.5 | 84586.3 |


|                     |   lag_1 |   lag_2 |
|:--------------------|--------:|--------:|
| 2025-04-21 00:00:00 | 86190.5 | 86190.5 |


|                     |   lag_1 |   lag_2 |
|:--------------------|--------:|--------:|
| 2025-04-22 00:00:00 | 86190.5 | 86190.5 |


|                     |   lag_1 |   lag_2 |
|:--------------------|--------:|--------:|
| 2025-04-23 00:00:00 | 86190.5 | 86190.5 |


,prediction_label
2025-04-19,86190.515625
2025-04-20,86190.515625
2025-04-21,86190.515625
2025-04-22,86190.515625
2025-04-23,86190.515625


# observations

using a recusrsive window for lagged features, the model is not seeing enough dynamic change and is therefore converging on the same value quickly.

To make this better, this will likely require feature engineering i.e 

- Time-based: day_of_week, day_of_month, month
- more time lags, 7, 30 days. But with the high autocorrelation we may end up with the same issue.
- Rolling stats: rolling_mean_3, rolling_std_5

TODO - Cite this!

# Future forecasting

## Direct multi-step forecasting

using PyCaret to optimise the model and sklearn's MultiOutput regressor to predict multiple future steps at once.

In [197]:
# utilise existing code for lagged features, consider 30 day lags

btc_5y_close_df_30_lag = btc_5y_close_df.copy()

# significant time periods (useful for the user) and hopefully saves memory+processing
for i in [1, 3, 5, 7, 14, 30]:
    btc_5y_close_df_30_lag[f'lag_{i}'] = btc_5y_close_df_30_lag['close'].shift(i)
    
btc_5y_close_df_30_lag.dropna(inplace=True)

btc_5y_close_df_30_lag.head()


,close,lag_1,lag_2,lag_3,lag_5,lag_7,lag_14,lag_30
timestamp,,,,,,,,
2020-05-22,9170.00,9068.65,9511.43,9775.53,9680.04,9316.42,9800.01,7125.14
2020-05-23,9179.15,9170.00,9068.65,9511.43,9733.93,9381.27,9539.40,7482.39
2020-05-24,8720.34,9179.15,9170.00,9068.65,9775.53,9680.04,8722.77,7505.00
2020-05-25,8900.35,8720.34,9179.15,9170.00,9511.43,9733.93,8561.52,7538.67
2020-05-26,8841.18,8900.35,8720.34,9179.15,9068.65,9775.53,8810.79,7693.10


In [198]:
import numpy as np
# create sequences of data, for now a 60 day window for a 30 day prediction, for multi-step prediciton
def create_sequences(data: pd.DataFrame, window=120, horizon=30, step=1):
    """
    Returns:
    
        a tuple of numpy arrays (x and y)
    """
    x = []
    y = []
    # extract the target values
    target = data['close'].values
    # get the feature cols
    feature_cols = [col for col in data.columns.to_list() if col != 'close']
    # extract the feature values
    features = data.loc[:,feature_cols].values
    # loop over the dataset
    for i in range(0, len(data) - window - horizon + 1, step):
        # get x, all features, given input window
        x_i = features[i : i + window]
        # print("x shape", x_i.shape)
        # target sequence (Next horizon values)
        y_i = target[i+window: i+window+horizon]
        # print("y shape", y_i.shape)
        # flatten the input array, append to x
        # print("x flat", x_i.flatten().shape)
        x.append(x_i.flatten())
        # append y array to list
        y.append(y_i)
    return np.array(x), np.array(y)

# test
X, y = create_sequences(btc_5y_close_df_30_lag)
print("x shape", X.shape)
print("y shape", y.shape)

x shape (1644, 840)
y shape (1644, 30)


In [199]:
# perfrom train test split

# Step 2: Split data for training/testing
# Use 80% for training, 20% for testing
split_idx = int(len(X) * 0.8)
X_train, X_test = X[:split_idx], X[split_idx:]
y_train, y_test = y[:split_idx], y[split_idx:]

print("x shape", X.shape)
print("y shape", y.shape)
print("X_test shape: ", X_test.shape)

x shape (1644, 840)
y shape (1644, 30)
X_test shape:  (329, 840)


In [200]:
from xgboost import XGBRegressor
from sklearn.multioutput import MultiOutputRegressor
# train and tune xgboost with pycaret using single step prediciton

# for this we need to ensure that the target data is only a one-step target.

def optimise_model(experiment: RegressionExperiment, custom_params:dict):
    final_xgb = experiment.create_model('xgboost',
                                        **custom_params)
    
    return final_xgb

def train_ts_xgboost_multi_step(data: pd.DataFrame, X: np.array, y: np.array): 
    experiment = RegressionExperiment().setup(
        data=data,
        target='close',
        data_split_shuffle=False,
        fold=5,
        fold_strategy='timeseries',
        session_id=456,
        train_size=0.8
    )
    
    xgb = experiment.create_model("xgboost")
    
    xgb_tuned = experiment.tune_model(xgb)
    xgb_params = xgb_tuned.get_params()
    
    # key hyperparameters
    # each tree sees more data, reducing variance
    xgb_params['subsample'] = 0.8
    # deeper trees to increase model complexity and ability to fit.
    xgb_params['max_depth'] = 4
    # reduce learning rate to increase regularisation
    xgb_params['learning_rate'] = 0.05
    
    # optimise the final model - mainly for scoring
    _ = optimise_model(experiment, xgb_params)
    
    X = X.astype(np.float32)
    y = y.astype(np.float32)

    base_model = XGBRegressor(**xgb_params)
    
    print("\n--- Training multi output XGB regressor ---")
    print("...")
    multi_model = MultiOutputRegressor(base_model).fit(X, y)
    
    return multi_model, xgb_params
    
    
# test
multi_step_xgb, params = train_ts_xgboost_multi_step(btc_5y_close_df_30_lag, X_train, y_train)


,Description,Value
0,Session id,456
1,Target,close
2,Target type,Regression
3,Original data shape,"(1793, 8)"
4,Transformed data shape,"(1793, 8)"
5,Transformed train set shape,"(1434, 8)"
6,Transformed test set shape,"(359, 8)"
7,Numeric features,7
8,Preprocess,True
9,Imputation type,simple


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,10579.7285,190898480.0000,13816.6016,-1.1862,0.3145,0.2047
1,2339.5132,9235188.0000,3038.9453,0.8683,0.0611,0.0482
2,2758.3975,9208641.0000,3034.5742,0.4609,0.1409,0.1369
3,1246.8951,2876734.5000,1696.0939,0.7528,0.0674,0.0474
4,2084.1516,11259663.0000,3355.5422,0.9464,0.0562,0.0387
Mean,3801.7372,44695741.3000,4988.3514,0.3684,0.1280,0.0952
Std,3424.7669,73155760.6662,4451.0775,0.7946,0.0983,0.0654


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,9733.5439,159061424.0000,12611.9561,-0.8216,0.2805,0.1893
1,1746.9001,5361965.5000,2315.5918,0.9235,0.0463,0.0362
2,1674.8158,4032282.2500,2008.0543,0.7640,0.0905,0.0811
3,686.5734,789918.8125,888.7737,0.9321,0.0348,0.0261
4,1839.1106,9328369.0000,3054.2378,0.9556,0.0501,0.0334
Mean,3136.1888,35714791.9125,4175.7227,0.5507,0.1005,0.0732
Std,3324.8716,61734037.9568,4275.2934,0.6895,0.0919,0.0612


Fitting 5 folds for each of 10 candidates, totalling 50 fits


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,10468.3447,184712784.0000,13590.9082,-1.1154,0.3080,0.2031
1,1890.5598,6487889.5000,2547.1335,0.9075,0.0495,0.0383
2,1794.8831,4140569.7500,2034.8390,0.7576,0.0930,0.0872
3,799.3998,1162708.5000,1078.2897,0.9001,0.0421,0.0305
4,1607.5997,7057441.0000,2656.5845,0.9664,0.0439,0.0294
Mean,3312.1574,40712278.5500,4381.5510,0.4832,0.1073,0.0777
Std,3598.7094,72030217.0508,4638.3499,0.8022,0.1021,0.0662



--- Training multi output XGB regressor ---
...


# Evaluate the multi-step regressor

In [202]:
# wrap tuned model with MultiOutputRegressor to predict multiple steps at once
y_pred = multi_step_xgb.predict(X_test)
# evaluate model
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, root_mean_squared_error

# this gives aggregate metrics across all test sequences and all prediction days
print(f"multi step model MAE: {mean_absolute_error(y_true= y_test[:,0], y_pred=y_pred[:, 0]):.2f}")
print(f"multi step model MSE: {mean_squared_error(y_true= y_test[:,0], y_pred=y_pred[:, 0]):.2f}")
print(f"multi step model R2: {r2_score(y_true= y_test[:,0], y_pred=y_pred[:, 0]):.2f}")

# eval_df = btc_5y_close_df_30_lag.loc[btc_5y_close_df_30_lag.index[-30:], ['close']]

# get the first day prediction for every sample
eval_df = pd.DataFrame()
eval_df['close'] = y_test[:,0]
eval_df['predictions'] = y_pred[:, 0]

eval_index = btc_5y_close_df_30_lag.index[-y_pred.shape[0]:]

fig = px.line(eval_df, x=eval_index, y=['close', 'predictions'], template='plotly_dark', labels={"value": "close price $"})
fig.show()


multi step model MAE: 16221.72
multi step model MSE: 465266510.30
multi step model R2: -0.75


# finalise the multi-step regressor

In [162]:
final_multi_xgb = multi_step_xgb.fit(X_test, y_test)

In [163]:
# forecast - use most recent 30 day window, generate predictions for next 30 days
def forecast(data: pd.DataFrame, model: MultiOutputRegressor, window=120, horizon=30) -> pd.DataFrame:
    """
    Remember, in this case we are predicting one multi-step interval, which requires
    one input sample.
    Therefore, we take the latest window of data (matching how the model was trained).
    Then, flatten these values to achieve a 1D array.
    Because scikit models require a 2d array (n_samples, n_features), we reshape.
    (1,-1) whcih means one row, automatically determine the column dimension aka features
    
    n.b there is an interdepenency with the create_sequences function, if the window
    or horizon is changed, the data sequences will also need to match this pattern.
    """
    latest_window = data[-window:].drop("close", axis=1).values.flatten().reshape(1,-1)
    forecast = model.predict(latest_window)[0] 
    last_date = data.index[-1]
    forecast_dates = pd.date_range(last_date + pd.Timedelta(days=1), periods=horizon)
    
    # create forecast df
    forecast_df = pd.DataFrame(
        {
            'timestamp': forecast_dates,
            'forecast': forecast
        }
    ).set_index('timestamp', drop=True)
    
    return forecast_df
# test
forecasts = forecast(btc_5y_close_df_30_lag, final_multi_xgb)

print(forecasts.head().to_markdown())

| timestamp           |   forecast |
|:--------------------|-----------:|
| 2025-04-19 00:00:00 |    84748.4 |
| 2025-04-20 00:00:00 |    85020.4 |
| 2025-04-21 00:00:00 |    83955   |
| 2025-04-22 00:00:00 |    84514.9 |
| 2025-04-23 00:00:00 |    84819.1 |


In [164]:
# plot most recent 30 days with 30 day forecast
last_month = btc_5y_close_df_30_lag.loc[btc_5y_close_df_30_lag.index[-60:], ['close']]

last_month_pred = pd.concat([last_month, forecasts])
# last_month_pred

fig = px.line(last_month_pred, x=last_month_pred.index, y=['close', 'forecast'], template='plotly_dark', labels={"value": "close price $"})
fig.add_vline(x=last_month.index[-1])
fig.show()

In [165]:
print("--- Tuned XGB Hyperparameters ---")
for key, value in params.items():
    print("{}:\n\t{}".format(key, value))

--- Tuned XGB Hyperparameters ---
objective:
	reg:squarederror
base_score:
	None
booster:
	gbtree
callbacks:
	None
colsample_bylevel:
	None
colsample_bynode:
	None
colsample_bytree:
	0.9
device:
	cpu
early_stopping_rounds:
	None
enable_categorical:
	False
eval_metric:
	None
feature_types:
	None
gamma:
	None
grow_policy:
	None
importance_type:
	None
interaction_constraints:
	None
learning_rate:
	0.05
max_bin:
	None
max_cat_threshold:
	None
max_cat_to_onehot:
	None
max_delta_step:
	None
max_depth:
	4
max_leaves:
	None
min_child_weight:
	1
missing:
	nan
monotone_constraints:
	None
multi_strategy:
	None
n_estimators:
	280
n_jobs:
	-1
num_parallel_tree:
	None
random_state:
	456
reg_alpha:
	10
reg_lambda:
	0.7
sampling_method:
	None
scale_pos_weight:
	28.5
subsample:
	0.8
tree_method:
	auto
validate_parameters:
	None
verbosity:
	0


# validate against other 5 yr coin data (selected coins from clusters)

In [174]:
selected_coins = [
    'BTCUSDT', 'BNBUSDT', 'ETHUSDT', 'LTCUSDT'
]

In [ ]:
def add_features(df):
    """Add technical indicators and features to the dataframe"""
    # Create a copy of the dataframe
    data = df.copy()

    # Add lagged features
    for lag in [1, 3, 5]:
        data[f'close_lag_{lag}'] = data['close'].shift(lag)

    # Add rolling stats
    data['ma7'] = data['close'].rolling(window=7).mean()
    data['ma30'] = data['close'].rolling(window=30).mean()
    data['std7'] = data['close'].rolling(window=7).std()

    # Add price changes
    data['price_change_1d'] = data['close'].pct_change(1)
    data['price_change_7d'] = data['close'].pct_change(7)

    # Calculate RSI (14-period)
    delta = data['close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / loss
    data['rsi'] = 100 - (100 / (1 + rs))

    # Drop rows with NaN values
    data = data.dropna()

    return data

def xgboost_train_pipeline_full(coin: str, df: pd.DataFrame, params: dict):
    # Step - drop columns
    df.drop(['symbol', 'open', 'high', 'low', 'volume'], axis=1, inplace=True)
    # step - feature engineering
    df = add_features(df)
    
    df.dropna(inplace=True)
    
    print(df.tail(2).to_markdown())
    
    # step 2: Create multi-step sequences of data (30 day prediction window)
    X, y = create_sequences(df)
    
    # Step 3: Split data for training/testing
    # Use 80% for training, 20% for testing
    split_idx = int(len(X) * 0.8)
    X_train, X_test = X[:split_idx], X[split_idx:]
    y_train, y_test = y[:split_idx], y[split_idx:]

    print("x shape", X.shape)
    print("y shape", y.shape)
    
    base_model = XGBRegressor(**params)
    
    # step 4:
    print("\n--- Training multi output XGB regressor ---")
    print("...")
    multi_model = MultiOutputRegressor(base_model).fit(X_train, y_train)
    
    # step 5
    print("\n---Evaluating model ---")
    print(f"--- {coin} ---")
    y_pred = multi_model.predict(X_test)

    print(f"multi step model MAE: {mean_absolute_error(y_true=y_test[:,0], y_pred=y_pred[:, 0]):.2f}")
    print(f"multi step model MSE: {mean_squared_error(y_true=y_test[:,0], y_pred=y_pred[:, 0]):.2f}")
    print(f"multi step model R2: {r2_score(y_true=y_test, y_pred=y_pred):.2f}")

    # get the first day prediction for every sample
    eval_df = pd.DataFrame()
    eval_df['close'] = y_test[:,0]
    eval_df['predictions'] = y_pred[:, 0]
    eval_index = btc_5y_close_df_30_lag.index[-y_pred.shape[0]:]


    fig = px.line(eval_df, x=eval_index, y=['close', 'predictions'], template='plotly_dark', labels={"value": "close price $"},
                  title=coin)
    fig.show()
        


In [191]:
for coin in selected_coins:
    df_filename = coin.lower() + "_1d.csv"
    df = pd.read_csv("../backend/data/"+df_filename, parse_dates=True, index_col=0)
    # use the params trained from the initial model on BTC 5 year lagged close prices.
    xgboost_train_pipeline_full(coin, df, params)
    

| timestamp           |   close |   close_lag_1 |   close_lag_3 |   close_lag_5 |     ma7 |    ma30 |    std7 |   price_change_1d |   price_change_7d |     rsi |
|:--------------------|--------:|--------------:|--------------:|--------------:|--------:|--------:|--------:|------------------:|------------------:|--------:|
| 2025-04-17 00:00:00 | 84947.9 |       84030.4 |       84591.6 |       85276.9 | 84239.2 | 83675.2 | 706.587 |        0.010919   |         0.0670869 | 52.9624 |
| 2025-04-18 00:00:00 | 84586.3 |       84947.9 |       83644   |       83760   | 84405.3 | 83599.8 | 613.481 |       -0.00425661 |         0.0139346 | 51.2022 |
x shape (1647, 1080)
y shape (1647, 30)

--- Training multi output XGB regressor ---
...

---Evaluating model ---
--- BTCUSDT ---
multi step model MAE: 24210.62
multi step model MSE: 825621244.14
multi step model R2: -2.20


| timestamp           |   close |   close_lag_1 |   close_lag_3 |   close_lag_5 |     ma7 |    ma30 |    std7 |   price_change_1d |   price_change_7d |     rsi |
|:--------------------|--------:|--------------:|--------------:|--------------:|--------:|--------:|--------:|------------------:|------------------:|--------:|
| 2025-04-17 00:00:00 |  590.49 |        582.8  |        584.54 |        597.29 | 586.407 | 600.053 | 5.82794 |       0.0131949   |        0.021556   | 49.1644 |
| 2025-04-18 00:00:00 |  590.54 |        590.49 |        579.52 |        584.28 | 587.066 | 599.114 | 6.02226 |       8.46754e-05 |        0.00786783 | 47.1423 |
x shape (1647, 1080)
y shape (1647, 30)

--- Training multi output XGB regressor ---
...

---Evaluating model ---
--- BNBUSDT ---
multi step model MAE: 141.15
multi step model MSE: 26788.53
multi step model R2: -6.54


| timestamp           |   close |   close_lag_1 |   close_lag_3 |   close_lag_5 |     ma7 |    ma30 |    std7 |   price_change_1d |   price_change_7d |     rsi |
|:--------------------|--------:|--------------:|--------------:|--------------:|--------:|--------:|--------:|------------------:|------------------:|--------:|
| 2025-04-17 00:00:00 | 1583.62 |       1577.14 |       1623.77 |       1644.18 | 1597.44 | 1787.64 | 27.3612 |        0.0041087  |         0.0403153 | 37.5028 |
| 2025-04-18 00:00:00 | 1588.27 |       1583.62 |       1588.78 |       1597.76 | 1600.5  | 1772.05 | 24.408  |        0.00293631 |         0.0136707 | 37.8267 |
x shape (1647, 1080)
y shape (1647, 30)

--- Training multi output XGB regressor ---
...

---Evaluating model ---
--- ETHUSDT ---
multi step model MAE: 361.15
multi step model MSE: 214700.79
multi step model R2: 0.36


| timestamp           |   close |   close_lag_1 |   close_lag_3 |   close_lag_5 |     ma7 |    ma30 |    std7 |   price_change_1d |   price_change_7d |     rsi |
|:--------------------|--------:|--------------:|--------------:|--------------:|--------:|--------:|--------:|------------------:|------------------:|--------:|
| 2025-04-17 00:00:00 |   74.95 |         74.33 |         77.18 |         78.62 | 76.3414 | 82.9277 | 1.59811 |        0.00834118 |         0.0154451 | 38.1219 |
| 2025-04-18 00:00:00 |   75.6  |         74.95 |         75.27 |         77.85 | 76.2571 | 82.3113 | 1.62279 |        0.00867245 |        -0.0077438 | 37.5811 |
x shape (1647, 1080)
y shape (1647, 30)

--- Training multi output XGB regressor ---
...

---Evaluating model ---
--- LTCUSDT ---
multi step model MAE: 10.41
multi step model MSE: 235.69
multi step model R2: 0.49


# Score comparisons

BTC baseline score

multi step model MAE: 24084.70

multi step model MSE: 817797760.33

multi step model R2: -2.17


## time lags, 3, 5

--- BNBUSDT ---

multi step model MAE: 141.69

multi step model MSE: 24791.47

multi step model R2: -6.07

--- ETHUSDT ---

multi step model MAE: 419.34

multi step model MSE: 295805.68

multi step model R2: 0.11

--- LTCUSDT ---

multi step model MAE: 12.23

multi step model MSE: 297.14

multi step model R2: 0.35


## additional features, rolling stats, pct change, and RSI (14-period)

### time lags 1, 3, 5

--- BNBUSDT ---

multi step model MAE: 141.15

multi step model MSE: 26788.53

multi step model R2: -6.54

--- ETHUSDT ---

multi step model MAE: 361.15

multi step model MSE: 214700.79

multi step model R2: 0.36

--- LTCUSDT ---

multi step model MAE: 10.41

multi step model MSE: 235.69

multi step model R2: 0.49


## additional features (as above), time lags 3, 5

--- BNBUSDT ---

multi step model MAE: 141.40

multi step model MSE: 26561.96

multi step model R2: -6.48

--- ETHUSDT ---

multi step model MAE: 370.47

multi step model MSE: 222145.35

multi step model R2: 0.33

--- LTCUSDT ---

multi step model MAE: 10.51

multi step model MSE: 238.89

multi step model R2: 0.49


## time lags 3

--- BNBUSDT ---
multi step model MAE: 141.56

multi step model MSE: 26632.66

multi step model R2: -6.50

--- ETHUSDT ---
multi step model MAE: 369.85
multi step model MSE: 221783.46
multi step model R2: 0.34


--- LTCUSDT ---
multi step model MAE: 10.49
multi step model MSE: 238.06
multi step model R2: 0.49

## time lags 5

--- BNBUSDT ---

multi step model MAE: 141.12

multi step model MSE: 26499.60

multi step model R2: -6.46

--- ETHUSDT ---

multi step model MAE: 369.05

multi step model MSE: 220872.28

multi step model R2: 0.34

--- LTCUSDT ---

multi step model MAE: 10.53

multi step model MSE: 239.03

multi step model R2: 0.48

## time lags 1, 3, 5, 7, 14, 30

--- BNBUSDT ---

multi step model MAE: 151.00

multi step model MSE: 30413.84

multi step model R2: -7.56

--- ETHUSDT ---

multi step model MAE: 373.46

multi step model MSE: 227514.94

multi step model R2: 0.32

--- LTCUSDT ---

multi step model MAE: 10.47

multi step model MSE: 233.79

multi step model R2: 0.50